# Face Recognition using SiameseNetwork

## 1. AntiSpoofing

In [ ]:
import dlib
import cv2
import numpy as np
import imutils
from imutils import face_utils
from scipy.spatial import distance as dist
from imutils.video import VideoStream
import time
import os

In [ ]:
# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold
EYE_AR_THRESH = 0.23 #baseline
EYE_AR_CONSEC_FRAMES = 3

# eye landmarks
eye_landmarks = "shape_predictor_68_face_landmarks.dat"
# initialize the frame counters and the total number of blinks
COUNTER = 0
TOTAL = 0

## 1.1 eye blink detector class

In [ ]:
class eye_blink_detector():
    def __init__(self):
        # cargar modelo para detecction frontal de rostros
        self.detector_faces = dlib.get_frontal_face_detector()
        # cargar modelo para deteccion de puntos de ojos
        self.predictor_eyes = dlib.shape_predictor(eye_landmarks)

    def eye_blink(self,gray,rect,COUNTER,TOTAL):
        (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
        (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = self.predictor_eyes(gray, rect)
        shape = face_utils.shape_to_np(shape)
        # extract the left and right eye coordinates, then use the
        # coordinates to compute the eye aspect ratio for both eyes
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = self.eye_aspect_ratio(leftEye)
        rightEAR = self.eye_aspect_ratio(rightEye)
        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2.0
        # check to see if the eye aspect ratio is below the blink
        # threshold, and if so, increment the blink frame counter
        if ear < EYE_AR_THRESH:
            COUNTER += 1
        # otherwise, the eye aspect ratio is not below the blink
        # threshold
        else:
            # if the eyes were closed for a sufficient number of
            # then increment the total number of blinks
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                TOTAL += 1
            # reset the eye frame counter
            COUNTER = 0
        return COUNTER,TOTAL

    def eye_aspect_ratio(self,eye):
        # compute the euclidean distances between the two sets of
        # vertical eye landmarks (x, y)-coordinates
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        # compute the euclidean distance between the horizontal
        # eye landmark (x, y)-coordinates
        C = dist.euclidean(eye[0], eye[3])
        # compute the eye aspect ratio
        ear = (A + B) / (2.0 * C)
        # return the eye aspect ratio
        return ear

## 1.2 eye blink detection function

In [ ]:
def convert_rectangles2array(rectangles,image):
    res = np.array([])
    for box in rectangles:
        [x0,y0,x1,y1] = max(0, box.left()), max(0, box.top()), min(box.right(), image.shape[1]), min(box.bottom(), image.shape[0])
        new_box = np.array([x0,y0,x1,y1])
        if res.size == 0:
            res = np.expand_dims(new_box,axis=0)
        else:
            res = np.vstack((res,new_box))
    return res

In [ ]:
def get_areas(boxes):
    areas = []
    for box in boxes:
        x0,y0,x1,y1 = box
        area = (y1-y0)*(x1-x0)
        areas.append(area)
    return areas

In [ ]:
def bounding_box(img,box,match_name=[]):
    for i in np.arange(len(box)):
        x0,y0,x1,y1 = box[i]
        img = cv2.rectangle(img,
                    (x0,y0),
                    (x1,y1),
                    (0,255,0),3);
        if not match_name:
            continue
        else:
            cv2.putText(img, match_name[i], (x0, y0-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
    return img

## 1.3 Get User's Verification images & Eye blinking Detection

In [ ]:
# instancio detector
detector = eye_blink_detector()
# iniciar variables para el detector de parapadeo
COUNTER = 0
TOTAL = 0
video_path = "data/videos/video.mp4" #/content/video.mp4
image_path = "facedata/realTimeData/input/"
verification_path = "facedata/realTimeData/verify/"

In [ ]:
def get_verification_images():
    
    verification_video_path = verification_path + "video/video.mp4"
    if not(os.path.isdir(verification_path + "video")):
        os.makedirs(verification_path + "video")
        
    cap = cv2.VideoCapture(0)
    # 저장될 영상은 mp4형식이다.
    # 캠이 여러대인 경우 인자로 0, 1, 2, 3...을 넣어주면 된다.
    # 캠으로부터 정보를 읽어들일 수 없는 경우 에러 메세지를 반환한다.
    if cap.isOpened() == False:
        print("Unable to read camera")

    # 캠으로부터 정보를 읽어들일 수 있으면,
    else:
        # 프레임의 정보 가져와 변수에 저장한다. 
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        print("원을 그리듯이 고개를 돌려주세요")

        #캠으로 들어온 비디오를 따로 저장한다.
        out = cv2.VideoWriter(verification_video_path,
                            cv2.VideoWriter_fourcc('D', 'I', 'V', 'X'),
                            10,
                            (frame_width, frame_height) )


        # 동영상은 사진을 여러장 이어서 보여주는 개념이다.
        # 1초에 몇 장의 이미지가 들어가는지, fps(frame per second) 단위를 쓴다.
        # 캠으로부터 이미지 한 장만을 받아올 게 아니므로, 반복문을 사용한다.
        start_time = time.time()
        while True:
            ret, frame = cap.read()
            if ret == True:
                out.write(frame)
                cv2.imshow('frame', frame)
                recording_time = time.time() - start_time
                #esc를 입력하면, 이미지를 받아오길 멈추게 한다.
                if cv2.waitKey(1) & 0xFF == 27: # esc누르면 종료
                    break
                elif recording_time > 10: #최대 10초 정도까지 촬영가능
                    print("time over")
                    break
            else:
                break

        # sql 커서와 커넥션을 다 사용하고 나면 연결을 닫아주듯이, 비디오캡쳐도 닫아준다.
        cap.release()
        # 파일도 더 이상 작성하지 않도록 한다.
        out.release()

        #화면에 띄운 창을 닫아준다.
        cv2.destroyAllWindows()
        return verification_video_path

In [ ]:
mode = input("Do you want User Registration? [y or n]: ")
if (mode == 'Y' or mode == 'y') or not(os.path.isdir(verification_path)):
    if not(os.path.isdir(verification_path)):
        os.makedirs(verification_path)
    verification_video_path = get_verification_images()
    verification_image_path = verification_path
    if not(os.path.isdir(verification_path)):
        os.makedirs(verification_path)
    vidcap = cv2.VideoCapture(verification_video_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    all_frame = vidcap.get(cv2.CAP_PROP_FRAME_COUNT) #영상 총 프레임 수
    multiplier = all_frame / 50 # 어떤 영상이든 50장 캡쳐하도록
    count = 1
    success = True
    while success:
        frameId = int(round(vidcap.get(1)))
        success,image = vidcap.read()
        if frameId % multiplier < 1 : # 몇 프레임 당 한번씩 캡쳐할지
            cv2.imwrite(verification_image_path + "image%d.jpg" %count, image)
            print("saved image image%d.jpg" % count)
            count += 1

In [ ]:
# ----------------------------- video -----------------------------
# Eye blinking detection & Login

vs = cv2.VideoCapture(0) #VideoStream(src=0).start()
antiSpoofing = True

frame_width = int(vs.get(3))
frame_height = int(vs.get(4))

#캠으로 들어온 비디오를 따로 저장한다.
out = cv2.VideoWriter(video_path,
                    cv2.VideoWriter_fourcc('D', 'I', 'V', 'X'),
                    20,
                    (frame_width, frame_height) )

start_time = time.time()
print("눈을 깜박여주세요\n")
while True:
    star_time = time.time()
    ret, im = vs.read()
    if ret == True:
        out.write(im)
        cv2.imshow('frame', im)
        im = cv2.flip(im, 1)
        im = imutils.resize(im, width=720)
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        # detectar_rostro    
        rectangles = detector.detector_faces(gray, 0)
        boxes_face = convert_rectangles2array(rectangles,im)
        if len(boxes_face)!=0:
            # seleccionar el rostro con mas area
            areas = get_areas(boxes_face)
            index = np.argmax(areas)
            rectangles = rectangles[index]
            boxes_face = np.expand_dims(boxes_face[index],axis=0)
            # blinks_detector
            COUNTER,TOTAL = detector.eye_blink(gray,rectangles,COUNTER,TOTAL)
            # agregar bounding box
            img_post = bounding_box(im,boxes_face,['blinks: {}'.format(TOTAL)])
            if TOTAL > 1 :
                break
        else:
            img_post = im 
        # visualizacion 
        end_time = time.time() - star_time    
        FPS = 1/end_time
        cv2.putText(img_post,f"FPS: {round(FPS,3)}",(10,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
        cv2.imshow('blink_detection',img_post)
        recording_time = time.time() - start_time
        if TOTAL > 1 : #한번 이상 눈깜박거림
            break
        elif recording_time > 7: #약 7초 내에 blink감지 안되면
            antiSpoofing = False
            print("time over")
            break

# sql 커서와 커넥션을 다 사용하고 나면 연결을 닫아주듯이, 비디오캡쳐도 닫아준다.
vs.release()
# 파일도 더 이상 작성하지 않도록 한다.
out.release()

#화면에 띄운 창을 닫아준다.
cv2.destroyAllWindows()

In [ ]:
if antiSpoofing:
    print("PASS")
    vidcap = cv2.VideoCapture(video_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    multiplier = fps*20
    count = 1
    success = True
    while success:
        frameId = int(round(vidcap.get(1)))
        success,image = vidcap.read()
        if frameId % multiplier < 1 :
            cv2.imwrite(image_path + "image%d.jpg" %count, image)
            print("saved image image%d.jpg" % count)
            count += 1
else:
    print("Spoofing")

In [ ]:
print(os.path.isfile(image_path + "1.jpg"))

## 2. Face Recognition

### Pip install

In [ ]:
!pip install tensorflow
!pip install pillow
!pip install tqdm
!pip install keras
!pip install numpy

In [ ]:
# from google.colab import drive
# import os
# drive.mount('/content/drive')

## 2.1 Data Load

In [ ]:
import os
import pickle

import tqdm
from PIL import Image
import numpy as np


class DataLoader(object):
    """
    Class for loading data from image files
    """

    def __init__(self, width, height, cells, data_path, output_path):
        """
        Proper width and height for each image.
        """
        self.width = width
        self.height = height
        self.cells = cells
        self.data_path = data_path
        self.output_path = output_path

    def _open_image(self, path):
        """
        Using the Image library we open the image in the given path. The path must lead to a .jpg file.
        We then resize it to 105x105 like in the paper (the dataset contains 250x250 images.)

        Returns the image as a numpy array.
        """
        image = Image.open(path)
        image = image.resize((self.width, self.height))
        data = np.asarray(image)
        data = np.array(data, dtype='float64')
        return data

    def convert_image_to_array(self, person, image_num, data_path, predict=False):
        """
        Given a person, image number and datapath, returns a numpy array which represents the image.
        predict - whether this function is called during training or testing. If called when training, we must reshape
        the images since the given dataset is not in the correct dimensions.
        """
        max_zeros = 4
        image_num = '0' * max_zeros + image_num
        image_num = image_num[-max_zeros:]
        #image_path = os.path.join(data_path, 'lfw2', person, f'{person}_{image_num}.jpg')
        image_path = os.path.join(data_path, 'kfaceData_detect', person, f'{person}_{image_num}.jpg')
        image_data = self._open_image(image_path)
        #print(f"Image data shape: {image_data.shape}") ## 밑밑줄에서 오류시 Image_data의 현재 array확인을 위해 추가함
        if not predict:
            image_data = image_data.reshape(self.width, self.height, self.cells)
        return image_data
    
    def convert_image_to_array_for_predictInput(self, person, image_num, data_path, predict=False):
        """
        Given a person, image number and datapath, returns a numpy array which represents the image.
        predict - whether this function is called during training or testing. If called when training, we must reshape
        the images since the given dataset is not in the correct dimensions.
        """
        max_zeros = 4
        image_num = '0' * max_zeros + image_num
        image_num = image_num[-max_zeros:]
        #image_path = os.path.join(data_path, 'lfw2', person, f'{person}_{image_num}.jpg')
        image_path = os.path.join(data_path, person, f'{person}_{image_num}.jpg')
        image_data = self._open_image(image_path)
        #print(f"Image data shape: {image_data.shape}") ## 밑밑줄에서 오류시 Image_data의 현재 array확인을 위해 추가함
        if not predict:
            image_data = image_data.reshape(self.width, self.height, self.cells)
        return image_data

    def load(self, set_name):
        """
        Writes into the given output_path the images from the data_path.
        dataset_type = train or test
        """
        file_path = os.path.join(self.data_path, 'splits', f'{set_name}.txt')
        print(file_path)
        print('Loading dataset...')
        x_first = []
        x_second = []
        y = []
        names = []
        with open(file_path, 'r') as file:
            lines = file.readlines()
        for line in tqdm.tqdm(lines):
            line = line.split()
            if len(line) == 4:  # Class 0 - non-identical
                names.append(line)
                first_person_name, first_image_num, second_person_name, second_image_num = line[0], line[1], line[2], \
                                                                                           line[3]
                first_image = self.convert_image_to_array(person=first_person_name,
                                                          image_num=first_image_num,
                                                          data_path=self.data_path)
                second_image = self.convert_image_to_array(person=second_person_name,
                                                           image_num=second_image_num,
                                                           data_path=self.data_path)
                x_first.append(first_image)
                x_second.append(second_image)
                y.append(0)
            elif len(line) == 3:  # Class 1 - identical
                names.append(line)
                person_name, first_image_num, second_image_num = line[0], line[1], line[2]
                first_image = self.convert_image_to_array(person=person_name,
                                                          image_num=first_image_num,
                                                          data_path=self.data_path)
                second_image = self.convert_image_to_array(person=person_name,
                                                           image_num=second_image_num,
                                                           data_path=self.data_path)
                x_first.append(first_image)
                x_second.append(second_image)
                y.append(1)
                
            elif len(line) == 2:  # verification
                names.append(line)
                first_image_num, second_image_num = line[0], line[1]
                first_image = self.convert_image_to_array_for_predictInput(person='input',
                                                          image_num=first_image_num,
                                                          data_path=self.data_path)
                second_image = self.convert_image_to_array(person='verify',
                                                           image_num=second_image_num,
                                                           data_path=self.data_path)
                x_first.append(first_image)
                x_second.append(second_image)
                y.append(2)
                
            elif len(line) == 1:
                print(f'line with a single value: {line}')
        print('Done loading dataset')
        with open(self.output_path, 'wb') as f:
            pickle.dump([[x_first, x_second], y, names], f)


print("Loaded data loader")

## 2.2 SiameseNetwork

In [ ]:
import os
import pickle
import random

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Input, Sequential, Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Lambda, BatchNormalization, Activation, \
    Dropout
from tensorflow.keras.regularizers import l2


class SiameseNetwork(object):
    def __init__(self, seed, width, height, cells, loss, metrics, optimizer, dropout_rate):
        """
        Seed - The seed used to initialize the weights
        width, height, cells - used for defining the tensors used for the input images
        loss, metrics, optimizer, dropout_rate - settings used for compiling the siamese model (e.g., 'Accuracy' and 'ADAM)
        """
        
        K.clear_session()
        self.load_file = None
        self.seed = seed
        self.initialize_seed()
        self.optimizer = optimizer

        # Define the matrices for the input images
        input_shape = (width, height, cells)
        left_input = Input(input_shape)
        right_input = Input(input_shape)

        # Get the CNN architecture as presented in the paper (read the readme for more information)
        model = self._get_architecture(input_shape)
        encoded_l = model(left_input)
        encoded_r = model(right_input)

        # Add a layer to combine the two CNNs
        L1_layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
        L1_siamese_dist = L1_layer([encoded_l, encoded_r])
        L1_siamese_dist = Dropout(dropout_rate)(L1_siamese_dist)

        # An output layer with Sigmoid activation function
        prediction = Dense(1, activation='sigmoid', bias_initializer=self.initialize_bias)(L1_siamese_dist)
        
        self.threshold = 0.9  # Set threshold for similarity percentage

        siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)
        self.siamese_net = siamese_net
        self.siamese_net.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    def initialize_seed(self):
        """
        Initialize seed all for environment
        """
        os.environ['PYTHONHASHSEED'] = str(self.seed)
        random.seed(self.seed)
        np.random.seed(self.seed)
        tf.random.set_seed(self.seed)

    def initialize_weights(self, shape, dtype=None):
        """
        Called when initializing the weights of the siamese model, uses the random_normal function of keras to return a
        tensor with a normal distribution of weights.
        """
        return K.random_normal(shape, mean=0.0, stddev=0.01, dtype=dtype, seed=self.seed)

    def initialize_bias(self, shape, dtype=None):
        """
        Called when initializing the biases of the siamese model, uses the random_normal function of keras to return a
        tensor with a normal distribution of weights.
        """
        return K.random_normal(shape, mean=0.5, stddev=0.01, dtype=dtype, seed=self.seed)

    def _get_architecture(self, input_shape):
        """
        Returns a Convolutional Neural Network based on the input shape given of the images. This is the CNN network
        that is used inside the siamese model. Uses parameters from the siamese one shot paper.
        """
        model = Sequential()
        model.add(
            Conv2D(filters=64,
                   kernel_size=(10, 10),
                   input_shape=input_shape,
                   kernel_initializer=self.initialize_weights,
                   kernel_regularizer=l2(2e-4),
                   name='Conv1'
                   ))
        model.add(BatchNormalization())
        model.add(Activation("relu"))
        model.add(MaxPooling2D())

        model.add(
            Conv2D(filters=128,
                   kernel_size=(7, 7),
                   kernel_initializer=self.initialize_weights,
                   bias_initializer=self.initialize_bias,
                   kernel_regularizer=l2(2e-4),
                   name='Conv2'
                   ))
        model.add(BatchNormalization())
        model.add(Activation("relu"))
        model.add(MaxPooling2D())

        model.add(
            Conv2D(filters=128,
                   kernel_size=(4, 4),
                   kernel_initializer=self.initialize_weights,
                   bias_initializer=self.initialize_bias,
                   kernel_regularizer=l2(2e-4),
                   name='Conv3'
                   ))
        model.add(BatchNormalization())
        model.add(Activation("relu"))
        model.add(MaxPooling2D())

        model.add(
            Conv2D(filters=256,
                   kernel_size=(4, 4),
                   kernel_initializer=self.initialize_weights,
                   bias_initializer=self.initialize_bias,
                   kernel_regularizer=l2(2e-4),
                   name='Conv4'
                   ))
        model.add(BatchNormalization())
        model.add(Activation("relu"))

        model.add(Flatten())
        model.add(
            Dense(4096,
                  activation='sigmoid',
                  kernel_initializer=self.initialize_weights,
                  kernel_regularizer=l2(2e-3),
                  bias_initializer=self.initialize_bias))
        return model

    def _load_weights(self, weights_file):
        """
        A function that attempts to load pre-existing weight files for the siamese model. If it succeeds then returns
        True and updates the weights, otherwise False.
        :return True if the file is already exists
        """
        self.siamese_net.summary()
        self.load_file = weights_file
        if os.path.exists(weights_file):  # if the file is already exists, load and return true
            print('Loading pre-existed weights file')
            self.siamese_net.load_weights(weights_file)
            return True
        return False

    def fit(self, weights_file, train_path, validation_size, batch_size, epochs, early_stopping, patience, min_delta):
        """
        Function for fitting the model. If the weights already exist, just return the summary of the model. Otherwise,
        perform a whole train/validation/test split and train the model with the given parameters.
        """
        with open(train_path, 'rb') as f:
            x_train, y_train, names = pickle.load(f)
        """
        X_train[0]:  |----------x_train_0---------------------------|-------x_val_0--------|
        X_train[1]:  |----------x_train_1---------------------------|-------x_val_1--------|
        y_train:     |----------y_train_0 = y_train_1---------------|----y_val_0=y_val_1---|
        """
        x_train_0, x_val_0, y_train_0, y_val_0 = train_test_split(x_train[0], y_train,
                                                                  test_size=validation_size,
                                                                  random_state=self.seed)
        x_train_1, x_val_1, y_train_1, y_val_1 = train_test_split(x_train[1], y_train,
                                                                  test_size=validation_size,
                                                                  random_state=self.seed)
        x_train_0 = np.array(x_train_0, dtype='float64')
        x_val_0 = np.array(x_val_0, dtype='float64')
        x_train_1 = np.array(x_train_1, dtype='float64')
        x_val_1 = np.array(x_val_1, dtype='float64')
        x_train = [x_train_0, x_train_1]
        x_val = [x_val_0, x_val_1]
        if y_train_0 != y_train_1 and y_val_0 != y_val_1:
            raise Exception("y train lists or y validation list do not equal")
        y_train_both = np.array(y_train_0, dtype='float64')
        y_val_both = np.array(y_val_0, dtype='float64')
        if not self._load_weights(weights_file=weights_file):
            print('No such pre-existed weights file')
            print('Beginning to fit the model')
            callback = []
            if early_stopping:
                """
                We used the EarlyStopping function monitoring on the validation loss with a minimum delta of 0.1
                (Minimum change in the monitored quantity to qualify as an improvement, i.e.
                an absolute change of less than min_delta, will count as no improvement.) and patience 5 
                (Number of epochs with no improvement after which training will be stopped.).
                The direction is automatically inferred from the name of the monitored quantity (‘auto’).
                """
                es = EarlyStopping(monitor='val_loss', min_delta=min_delta, patience=patience, mode='auto', verbose=1)
                callback.append(es)
            self.siamese_net.fit(x_train, y_train_both, batch_size=batch_size, epochs=epochs,
                                 validation_data=(x_val, y_val_both), callbacks=callback, verbose=1)
            self.siamese_net.save_weights(self.load_file)
        # evaluate on the testing set
        loss, accuracy = self.siamese_net.evaluate(x_val, y_val_both, batch_size=batch_size)
        print(f'Loss on Validation set: {loss}')
        print(f'Accuracy on Validation set: {accuracy}')

    def evaluate(self, test_file, batch_size, analyze=False):
        """
        Function for evaluating the final model after training.
        test_file - file path to the test file.
        batch_size - the batch size used in training.

        Returns the loss and accuracy results.
        """
        with open(test_file, 'rb') as f:
            x_test, y_test, names = pickle.load(f)
        print(f'Available Metrics: {self.siamese_net.metrics_names}')
        y_test = np.array(y_test, dtype='float64')
        x_test[0] = np.array(x_test[0], dtype='float64')
        x_test[1] = np.array(x_test[1], dtype='float64')
        # evaluate on the test set
        loss, accuracy = self.siamese_net.evaluate(x_test, y_test, batch_size=batch_size)
        if analyze:
            self._analyze(x_test, y_test, names)
        return loss, accuracy
    
    def predict(self, verify_file):
        with open(verify_file, 'rb') as f:
            x_verify, y, names = pickle.load(f)
        print('\nprediction')

        x_verify[0] = np.array(x_verify[0], dtype='float64')
        x_verify[1] = np.array(x_verify[1], dtype='float64')
        y_pred = self.siamese_net.predict(x_verify)
        return y_pred

    def _analyze(self, x_test, y_test, names):
        """
        Function used for evaluating our network in the methods proposed in the assignment.
        We will find:
        - The person who has 2 images that are the most dissimilar to each other
        - The person with the two images that are the most similar to each other
        - Two people with the most dissimilar images, and
        - The two people with the most similar images.
        """
        best_class_0_prob = 1  # correct classification for different people, y=0, prediction->0
        best_class_0_name = None
        worst_class_0_prob = 0  # misclassification for different people, y=0, prediction->1
        worst_class_0_name = None
        best_class_1_prob = 0  # correct classification for same people, y=1, prediction->1
        best_class_1_name = None
        worst_class_1_prob = 1  # misclassification for same people, y=1, prediction->0
        worst_class_1_name = None
        prob = self.siamese_net.predict(x_test)
        for pair_index in range(len(names)):
            name = names[pair_index]
            y_pair = y_test[pair_index]
            pair_prob = prob[pair_index][0]
            if y_pair == 0:  # different people (actual)
                if pair_prob < best_class_0_prob:  # correct classification for different people, y=0, prediction->0
                    best_class_0_prob = pair_prob
                    best_class_0_name = name
                if pair_prob > worst_class_0_prob:  # misclassification for different people, y=0, prediction->1
                    worst_class_0_prob = pair_prob
                    worst_class_0_name = name
            else:  # the same person (actual)
                if pair_prob > best_class_1_prob:  # correct classification for same people, y=1, prediction->1
                    best_class_1_prob = pair_prob
                    best_class_1_name = name
                if pair_prob < worst_class_1_prob:  # misclassification for same people, y=1, prediction->0
                    worst_class_1_prob = pair_prob
                    worst_class_1_name = name

        print(f'correct classification for different people, y=0, prediction->0, name: {best_class_0_name} | prob: {best_class_0_prob}')
        print(f'misclassification for different people, y=0, prediction->1, name: {worst_class_0_name} | prob: {worst_class_0_prob}')
        print(f'correct classification for same people, y=1, prediction->1, name: {best_class_1_name} | prob: {best_class_1_prob}')
        print(f'misclassification for same people, y=1, prediction->0, name: {worst_class_1_name} | prob: {worst_class_1_prob}')


print("Loaded Siamese Network")

## 2.3 Training, Testing, RealTime Test

In [ ]:
import os
import random
import time

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

path_separator = os.path.sep
# Environment settings
IS_COLAB = (os.name == 'posix')
LOAD_DATA = not (os.name == 'posix')
IS_EXPERIMENT = False ## False
train_name = 'train'
test_name = 'test'
verify_name = 'verify' #RealTime Test용
WIDTH = HEIGHT = 105
CEELS = 3 #흑백사진(gray scaling 한것):1, koreanface처럼 컬러사진:3
loss_type = "binary_crossentropy"
validation_size = 0.2
early_stopping = True
VERIFY = True #RealTime Test 여부

if IS_COLAB:
    # the google drive folder we used
    data_path = os.path.sep + os.path.join('content', 'drive', 'My\ Drive', 'datasets', 'lfw2').replace('\\', '')
else:
    # locally
    from data_loader import DataLoader
    from siamese_network import SiameseNetwork

    #data_path = os.path.join('lfwa', 'lfw2')
    data_path = "faceData"
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def run_combination(l, bs, ep, pat, md, seed, train_path, test_path):
    """
    This function gets the parameters and run the experiment.
    :return: loss - loss on the testing set, accuracy - accuracy on the testing set
    """
    # file types
    model_save_type = 'h5'
    # files paths
    initialize_seed(seed)
    parameters_name = f'seed_{seed}_lr_{l}_bs_{bs}_ep_{ep}_val_{validation_size}_' \
                      f'es_{early_stopping}_pa_{pat}_md_{md}'
    print(f'Running combination with {parameters_name}')
    # A path for the weights
    load_weights_path = os.path.join(data_path, 'weights', f'weights_{parameters_name}.{model_save_type}')

    siamese = SiameseNetwork(seed=seed, width=WIDTH, height=HEIGHT, cells=CEELS, loss=loss_type, metrics=['accuracy'],
                             optimizer=Adam(lr=l), dropout_rate=0.4)
    siamese.fit(weights_file=load_weights_path, train_path=train_path, validation_size=validation_size,
                batch_size=bs, epochs=ep, early_stopping=early_stopping, patience=pat,
                min_delta=md)
    loss, accuracy = siamese.evaluate(test_file=test_path, batch_size=bs, analyze=True)

    print(f'Loss on Testing set: {loss}')
    print(f'Accuracy on Testing set: {accuracy}')
    
    # predict_pairs(model) RealTime Test
    if VERIFY:
        data_set_save_type = 'pickle'
        verify_file = os.path.join(data_path, f'{verify_name}.{data_set_save_type}')  # A path for the train file
        verify_path = os.path.join('faceData','realTimeData')
        if LOAD_DATA:  # If the verify data already exists
            loader = DataLoader(width=WIDTH, height=HEIGHT, cells=CEELS, data_path=verify_path, output_path=verify_file)
            loader.load(set_name=verify_name)
        y_pred = siamese.predict(verify_file)
        print(y_pred)
            
    return loss, accuracy


def run():
    """
    The main function that runs the training and experiments. Uses the global variables above.
    """
    # file types
    data_set_save_type = 'pickle'
    train_path = os.path.join(data_path, f'{train_name}.{data_set_save_type}')  # A path for the train file
    test_path = os.path.join(data_path, f'{test_name}.{data_set_save_type}')  # A path for the test file
    if LOAD_DATA:  # If the training data already exists
        loader = DataLoader(width=WIDTH, height=HEIGHT, cells=CEELS, data_path=data_path, output_path=train_path)
        loader.load(set_name=train_name)
        loader = DataLoader(width=WIDTH, height=HEIGHT, cells=CEELS, data_path=data_path, output_path=test_path)
        loader.load(set_name=test_name)

    result_path = os.path.join(data_path, f'results.csv')  # A path for the train file
    results = {'lr': [], 'batch_size': [], 'epochs': [], 'patience': [], 'min_delta': [], 'seed': [], 'loss': [],
               'accuracy': []}
    for l in lr:
        for bs in batch_size:
            for ep in epochs:
                for pat in patience:
                    for md in min_delta:
                        for seed in seeds:
                            siamese, loss, accuracy = run_combination(l=l, bs=bs, ep=ep, pat=pat, md=md, seed=seed,
                                                             train_path=train_path, test_path=test_path)
                            results['lr'].append(l)
                            results['batch_size'].append(bs)
                            results['epochs'].append(ep)
                            results['patience'].append(pat)
                            results['min_delta'].append(md)
                            results['seed'].append(seed)
                            results['loss'].append(loss)
                            results['accuracy'].append(accuracy)
    df_results = pd.DataFrame.from_dict(results)
    df_results.to_csv(result_path)


def initialize_seed(seed):
    """
    Initialize all relevant environments with the seed.
    """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)


if __name__ == '__main__':
    if IS_EXPERIMENT:
        # Experiments settings
        seeds = [0]
        lr = [0.00005]
        batch_size = [32]
        epochs = [10]
        patience = [5]
        min_delta = [0.1]
    else:
        # Final settings
        seeds = [0]
        lr = [0.00005]
        batch_size = [32]
        epochs = [10]
        patience = [5]
        min_delta = [0.1]

    print(os.name)
    start_time = time.time()
    print('Starting the experiments')
    run()
    print(f'Total Running Time: {time.time() - start_time}')